In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# print(openai.Engine.list())

def print_lines(text, max_line_length=120):
    lines = []
    line = ""
    for word in text.split():
        if len(line + word) > max_line_length:
            lines.append(line)
            line = ""
        line += word + " "
    lines.append(line)

    for line in lines:
        print(line)

class ChatEngine:
    model = ""
    temp = 0

    def __init__(self, model="gpt-3.5-turbo", temp=0):
        self.model = model
        self.temp = temp

class Chat:
    messages: list = []
    engine: ChatEngine

    def __init__(self, engine, system_message, user_messages = []):
        self.engine = engine
        self.messages.append({"role": "system", "content": system_message})
        self.messages.extend([{"role": "user", "content": message} for message in user_messages])

    def get_answer(self, user_message):
        self.messages.append({"role": "user", "content": user_message})
        response = openai.ChatCompletion.create(
            model=self.engine.model,
            messages=self.messages,
            temperature=self.engine.temp,
        )
        answer = response.choices[0].message["content"]
        self.messages.append({"role": "assistant", "content": answer})
        return answer


In [ ]:
politeness_prompt = """You are a tone analyzer robot. Given the user message, \
you should rate its politeness on a scale of 1 to 10, \
where 1 is very rude and 10 is very polite. \ 
- You should answer with one number and nothing else"""

def politeness_score(prompt) -> int:
    engine = ChatEngine()
    chat = Chat(engine, politeness_prompt)
    answer = chat.get_answer(prompt)
    if not answer.isdigit():
        answer = 5
    return int(answer)

print(politeness_score("Hey!"))